In [1]:
!pip install faker
!pip install uszipcode

import random
import numpy as np
from faker import Faker
import pandas as pd
import csv
import zipfile
import scipy.stats

from datetime import datetime
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=False)
fake = Faker()

In [2]:
df_tax = pd.read_csv("codoxo_specialties_table (5).csv")

df_tax['id|specialty|taxonomy_codes'] = df_tax['id|specialty|taxonomy_codes'].str.split('|')

def getLastValue(aList):
    return aList[-1]

df_tax['id|specialty|taxonomy_codes'] = df_tax['id|specialty|taxonomy_codes'].apply(getLastValue)
df_tax['id|specialty|taxonomy_codes'] = df_tax['id|specialty|taxonomy_codes'].str.split('@')

tax = df_tax.set_index('0')['id|specialty|taxonomy_codes'].to_dict()




def syn_data(taxonomy, num_providers,  folder):
    num_claims_csv = folder+"/num_claim_df.csv"
    all_combo_csv = folder+"/all_combinations_age.csv"
    location_csv = folder+"/location_data.csv"
    c_amt_csv = folder+"/c.csv"
    d_amt_csv = folder+"/d.csv"
    
    
    
    rendering_df = pd.DataFrame()
    rendering_df["RenderingProviderEntityType"] = np.random.choice(["I", "O"], num_providers, p=[0.5, 0.5])
    rendering_df['RenderingProviderFName'] = [fake.first_name() for i in range(num_providers)]
    rendering_df['RenderingProviderMName'] = [fake.first_name() for i in range(num_providers)]
    rendering_df['RenderingProviderLName'] = [fake.last_name() for i in range(num_providers)]
    rendering_df['RenderingProviderPhoneNumber'] = [fake.msisdn() for i in range(num_providers)]
    rendering_df['RenderingProviderTIN']= np.random.randint(100000000,999999999, size=num_providers)
    rendering_df["RenderingProviderID"] = np.random.randint(10000,999999, size=num_providers)
    rendering_df["RenderingProviderID"] = rendering_df["RenderingProviderID"].astype(str)
    rendering_df['1'] = [fake.building_number() for i in range(num_providers)]
    rendering_df["2"] = [fake.street_name() for i in range(num_providers)]
    rendering_df['RenderingProviderAddressLine1'] = rendering_df["1"]+" "+rendering_df["2"]
    rendering_df["1"] = np.random.choice(["Suite", "Apt.", "Null"], num_providers, p=[0.1, 0.3, 0.6])
    rendering_df["2"] = [fake.building_number() for i in range(num_providers)]
    rendering_df["RenderingProviderAddressLine2"] = rendering_df["1"]+" "+rendering_df["2"]
    rendering_df.loc[rendering_df['RenderingProviderAddressLine2'].str.contains('Null', case=False), 'RenderingProviderAddressLine2'] = 'Null'
    rendering_df = rendering_df.drop(columns=['1',"2"])
    rendering_df['RenderingProviderOrg']= np.random.choice(["I", "O"], num_providers, p=[0.5, 0.5])
    rendering_df['RenderingProviderPhoneNumber'] = rendering_df['RenderingProviderPhoneNumber'].astype(str)
    rendering_df['RenderingProviderTIN'] = rendering_df['RenderingProviderTIN'].astype(str)
    rendering_df["CCN"] = np.random.randint(0,38000, size=num_providers)
    rendering_df["CCN"] = rendering_df["CCN"].apply(lambda x: '{0:0>5}'.format(x))
    rendering_df["RenderingProviderNPI"] = np.random.randint(10000,999999, size=num_providers)
    rendering_df["RenderingProviderNPI"] = rendering_df["RenderingProviderNPI"].astype(str)
    rendering_df['ren_id'] = range(1, 1+len(rendering_df))
    rendering_df['RenderingProviderTaxonomy'] = np.random.choice(tax[taxonomy], num_providers)
    
    
    
    #Probability of number of claims per provider
    #We'll eventually bring inference (of number of claims for each provider) from all customers for each specialty.
    #Based on MonteCarlo simulation from the empirical data
    num_claims_df = pd.read_csv(num_claims_csv)
    
    #How many providers do we want to synthesize?
    ## User defined for each specialty
    num_iter = 100000
    num_claims_avg_lst = []
    for ii in range(0, num_iter):
        num_claims_avg_lst.append(round(num_claims_df.sample(round(0.1*len(num_claims_df))).num_claims.mean()))
    dist_mean = np.round(np.mean(num_claims_avg_lst))
    dist_std = np.round(np.std(num_claims_avg_lst))
    group = [int(round(x)) for x in np.random.normal(loc=dist_mean, scale=3*dist_std,size=num_providers)]
    scipy.stats.describe(group)
    
    ### Determine probability of different combinations 
    ### We'll eventually bring inference (of number of claims for each combination) from all customers for each specialty.
    all_combinations = pd.read_csv(all_combo_csv)
    sel_combinations = all_combinations[all_combinations.prob_claims>=0.01]
    
    
    ## Unique combinations and corresponding weights
    comb_list = sel_combinations.drop(['num_claims', 'prob_claims'], axis = 1).values
    weights = sel_combinations['prob_claims'].tolist()
    
    final_df_list = []
    random_outliers = random.sample(range(0, num_providers), int(round(0.01*num_providers)))
    for ii in range(0, num_providers):
        if ii in random_outliers:
            tmp = pd.DataFrame(random.choices(comb_list, random.shuffle(weights), k=group[ii-1]), 
                           columns = sel_combinations.drop(['num_claims', 'prob_claims'], axis = 1).columns)
        #else:
        #    tmp = pd.DataFrame(random.choices(comb_list, random.shuffle(weights), k=group[ii]), columns = sel_combinations.drop(['num_claims', 'prob_claims'], axis = 1).columns)
        else:
            tmp = pd.DataFrame(random.choices(comb_list, weights, k=group[ii]),
                               columns = sel_combinations.drop(['num_claims', 'prob_claims'], axis = 1).columns)
        
        tmp['ren_id'] = str(ii+1)
        tmp['prov_claim_id'] = ['claim_' + str(x) for x in range(0, group[ii])]
        final_df_list.append(tmp)
    final_df = pd.concat(final_df_list)
    
    final_df["combos"] = final_df["combos"].apply(lambda x: x.split(":"))
    final_df['ClaimID'] = range(1, 1+len(final_df))
    
    
    
    pat_df = pd.DataFrame()
    
    pat_df["PatientState"] = [fake.state() for i in range(num_providers)]
    pat_df['PatientFName'] = [fake.first_name() for i in range(num_providers)]
    #######
    pat_df["pat_gender"] = np.random.choice(["Male", "Female", "Other"], pat_df.shape[0], p=[0.49, 0.49, 0.02])
    pat_df["age_bin"] = np.random.choice(["Adult", "Elderly", "Minor"], pat_df.shape[0], p=[0.4, 0.4, 0.2])
    ######
    pat_df["PatientLName"] = [fake.last_name() for i in range(num_providers)]
    pat_df['PatientMName'] = [fake.first_name() for i in range(num_providers)]
    pat_df["PatientID"] = np.random.randint(1000000000, 9999999999, size=num_providers)
    pat_df["PatientID"] = pat_df["PatientID"].astype(str)
    pat_df['1'] = [fake.building_number() for i in range(num_providers)]
    pat_df["2"] = [fake.street_name() for i in range(num_providers)]
    pat_df['PatientAddressLine1'] = pat_df["1"]+" "+pat_df["2"]
    pat_df["1"] = np.random.choice(["Suite", "Apt.", "Null"], num_providers, p=[0.1, 0.3, 0.6])
    pat_df["2"] = [fake.building_number() for i in range(num_providers)]
    pat_df["PatientAddressLine2"] = pat_df["1"]+" "+pat_df["2"]
    pat_df.loc[pat_df['PatientAddressLine2'].str.contains('Null', case=False), 'PatientAddressLine2'] = 'Null'
    pat_df["SubscriberState"]= pat_df["PatientState"]
    pat_df["SubscriberID"]= np.random.randint(1000000000, 9999999999, size=num_providers)
    pat_df["SubscriberID"]=pat_df["SubscriberID"].astype(str)
    pat_df["PatientRelationship"] = np.random.choice(["Dependant", "Spouse", "Subscriber"], pat_df.shape[0], p=[0.2, 0.4, 0.4])
    
    
    adult_df = pat_df.loc[(pat_df['age_bin'] == 'Adult')]
    elderly_df = pat_df.loc[(pat_df['age_bin'] == 'Elderly')]
    minor_df = pat_df.loc[(pat_df['age_bin'] == 'Minor')]
    
    adult_df["PatientDOB"] = [fake.date_of_birth(minimum_age= 22, maximum_age=65) for i in range(adult_df.shape[0])]
    elderly_df["PatientDOB"] = [fake.date_of_birth(minimum_age= 65, maximum_age=99) for i in range(elderly_df.shape[0])]
    minor_df["PatientDOB"] = [fake.date_of_birth(minimum_age= 1, maximum_age=18) for i in range(minor_df.shape[0])]
    
    adult_df["PatientRelationship"] = np.random.choice(["Dependant", "Spouse", "Subscriber"], adult_df.shape[0], p=[0.2, 0.4, 0.4])
    elderly_df["PatientRelationship"] = np.random.choice(["Spouse", "Subscriber"], elderly_df.shape[0], p=[0.5, 0.5])
    minor_df["PatientRelationship"] = "Dependant"
    pat_df = pd.concat([adult_df, elderly_df, minor_df])
    
    Sub_df = pat_df.loc[(pat_df['PatientRelationship'] == 'Subscriber')]
    NS_df = pat_df.loc[(pat_df['PatientRelationship'] != 'Subscriber')]
    
    NS_df["SubscriberGender"] = np.random.choice(["Male", "Female", "Other"], NS_df.shape[0], p=[0.49, 0.49, 0.02])
    NS_df['SubscriberDOB'] = [fake.date_of_birth(minimum_age= 22, maximum_age=99) for i in range(NS_df.shape[0])]
    
    NS_df['1'] = [fake.building_number() for i in range(NS_df.shape[0])]
    NS_df["2"] = [fake.street_name() for i in range(NS_df.shape[0])]
    NS_df['SubscriberAddressLine1'] = NS_df["1"]+" "+NS_df["2"]
    NS_df["1"] = np.random.choice(["Suite", "Apt.", "Null"], NS_df.shape[0], p=[0.1, 0.3, 0.6])
    NS_df["2"] = [fake.building_number() for i in range(NS_df.shape[0])]
    NS_df["SubscriberAddressLine2"] = NS_df["1"]+" "+NS_df["2"]
    NS_df.loc[NS_df['SubscriberAddressLine2'].str.contains('Null', case=False), 'SubscriberAddressLine2'] = 'Null'
    NS_df['SubscriberLName'] = NS_df['PatientLName']
    NS_df['SubscriberFName'] = [fake.first_name() for i in range(NS_df.shape[0])]
    NS_df['SubscriberMName'] = [fake.first_name() for i in range(NS_df.shape[0])]
    NS_df = NS_df.drop(columns = ["1","2"])
    #####
    
    Sub_df["SubscriberGender"] = Sub_df["pat_gender"]
    Sub_df['SubscriberDOB'] = Sub_df['PatientDOB']
    Sub_df['SubscriberAddressLine1'] = Sub_df['PatientAddressLine1']
    Sub_df["SubscriberAddressLine2"] = Sub_df["PatientAddressLine2"]
    Sub_df['SubscriberLName'] = Sub_df['PatientLName']
    Sub_df['SubscriberFName'] = Sub_df['PatientFName']
    Sub_df['SubscriberMName'] = Sub_df['PatientMName']
    
    pat_df = pd.concat([Sub_df, NS_df])
    pat_df = pat_df.drop(columns=['1',"2"])
    
    pat_df["PatientDOB"]=pat_df["PatientDOB"].astype(str)
    pat_df["SubscriberDOB"]=pat_df["SubscriberDOB"].astype(str)
    pat_df["Pat"]= pat_df['PatientState']+"@"+pat_df['PatientFName']+"@"+pat_df['pat_gender']+"@"+pat_df['PatientLName']+"@"+pat_df['PatientMName']+"@"+pat_df['PatientID']+"@"+pat_df['PatientAddressLine1']+"@"+pat_df['PatientAddressLine2']+"@"+pat_df['SubscriberState']+"@"+pat_df['SubscriberID']+"@"+pat_df['PatientRelationship']+"@"+pat_df['PatientDOB']+"@"+pat_df['SubscriberGender']+"@"+pat_df['SubscriberDOB']+"@"+pat_df['SubscriberAddressLine1']+"@"+pat_df['SubscriberAddressLine2']+"@"+pat_df['SubscriberLName']+"@"+pat_df['SubscriberFName']+"@"+pat_df['SubscriberMName']
    
    
    adult_df = pat_df.loc[(pat_df['age_bin'] == 'Adult')]
    elderly_df = pat_df.loc[(pat_df['age_bin'] == 'Elderly')]
    minor_df = pat_df.loc[(pat_df['age_bin'] == 'Minor')]
    
    adult_list = adult_df["Pat"].tolist()
    
    elderly_list = elderly_df["Pat"].tolist()
    
    minor_list = minor_df["Pat"].tolist()
    del pat_df
    del Sub_df
    del NS_df
    del adult_df
    del elderly_df
    del minor_df
    
    zip_length = rendering_df.shape[0] + 10
    
    zip_df =pd.read_csv(location_csv)
    zip_combinations = zip_df[zip_df.prob_combo>=0.0001]
    
    comb_list = zip_combinations.drop(['prob_combo'], axis = 1).values
    weights = zip_combinations['prob_combo'].tolist()
    
    zip_df = pd.DataFrame(random.choices(comb_list, weights,
                                         k=zip_length), columns = zip_df.drop(['prob_combo'], axis = 1).columns)
    
    rend_zips = zip_df['renderingproviderzipcode'].tolist()
    rendering_df['renderingproviderzipcode'] = np.random.choice(rend_zips, rendering_df.shape[0])
    
    rend_zip_df = rendering_df.merge(zip_df.drop_duplicates(), on='renderingproviderzipcode')
    
    dfRendChoices = pd.DataFrame()
    rend_tin = rend_zip_df['RenderingProviderTIN'].unique()
    i = 0
    for tin in rend_tin:
        comb_list = rend_zip_df[rend_zip_df['RenderingProviderTIN'] == tin].values
    
        dfTmp = pd.DataFrame(random.choices(comb_list,
                                            k=random.choices([1, 2, 3,4,5], k=1)[0]),
                             columns = rend_zip_df.columns)
        i+=1
        dfTmp["ren_id"] = str(i)
        dfRendChoices = dfRendChoices.append(dfTmp)
    dfRendChoices = dfRendChoices.drop_duplicates()
    del rend_zip_df
    del zip_df
    
    def zcode(x):
        return search.by_zipcode(x).city
    
    dfRendChoices['subscriberzipcode'] = dfRendChoices['subscriberzipcode'].astype(str)
    dfRendChoices['renderingproviderzipcode'] =dfRendChoices['renderingproviderzipcode'].astype(str)
    dfRendChoices['patientzipcode']=dfRendChoices['patientzipcode'].astype(str)
    dfRendChoices['billingproviderzipcode']=dfRendChoices['billingproviderzipcode'].astype(str)
    
    dfRendChoices['SubscriberCity'] = dfRendChoices['subscriberzipcode'].str[:5]
    dfRendChoices['RenderingProviderCity'] = dfRendChoices['renderingproviderzipcode'].str[:5]
    dfRendChoices['PatientCity'] = dfRendChoices['patientzipcode'].str[:5]
    dfRendChoices['BillingProviderCity'] = dfRendChoices['billingproviderzipcode'].str[:5]
    
    dfRendChoices['BillingProviderCity'] = dfRendChoices['BillingProviderCity'].str.replace('.','')
    dfRendChoices['PatientCity'] = dfRendChoices['PatientCity'].str.replace('.','')
    dfRendChoices['SubscriberCity'] =dfRendChoices['SubscriberCity'].str.replace('.','')
    dfRendChoices['RenderingProviderCity'] = dfRendChoices['RenderingProviderCity'].str.replace('.','')
    
    dfRendChoices['BillingProviderCity'] = dfRendChoices['BillingProviderCity'].astype(float)
    dfRendChoices['PatientCity'] = dfRendChoices['PatientCity'].astype(float)
    dfRendChoices['SubscriberCity'] = dfRendChoices['SubscriberCity'].astype(float)
    dfRendChoices['RenderingProviderCity'] = dfRendChoices['RenderingProviderCity'].astype(float)
    
    dfRendChoices['SubscriberCity'] = dfRendChoices['SubscriberCity'].fillna(0).astype(int).apply(zcode)
    dfRendChoices['RenderingProviderCity'] = dfRendChoices['RenderingProviderCity'].fillna(0).astype(int).apply(zcode)
    dfRendChoices['PatientCity'] = dfRendChoices['PatientCity'].fillna(0).astype(int).apply(zcode)
    dfRendChoices['BillingProviderCity'] = dfRendChoices['BillingProviderCity'].fillna(0).astype(int).apply(zcode)
    dfRendChoices.rename(columns = {'billingproviderzipcode':'BillingProviderZipCode', "renderingproviderzipcode": "RenderingProviderZipCode",
                               "pickupzipcode":"PickUpZipCode", "dropoffzipcode":"DropOffZipCode",
                               "prescribingproviderzipcode":"PrescribingProviderZipCode","subscriberzipcode":"SubscriberZipCode", "patientzipcode":"PatientZipCode","billingproviderstate":"BillingProviderState", 'renderingproviderstate':"RenderingProviderState", 'subscriberstate':"SubscriberState", 'prescribingproviderstate':"PrescribingProviderState" }, inplace = True)
    
    rend = dfRendChoices.values.tolist()
    
    out = []
    c =[]
    f= ""
    for a in rend:
        if f == a[12]:
            c.append(a)
        else:
            out.append(c)
            c = [a]
            f = (a[12]) 
    
    out.append(c)
    out.pop(0)
    ##
    final_df["Ren"] = final_df["ren_id"].apply(lambda x: random.choice(out[int(x)-2]))
    ###
    rend = (list(dfRendChoices.columns))
    del dfRendChoices
    
    
    num_lines= final_df.shape[0]
    
    billing_df = pd.DataFrame()
    billing_df["BillingProviderID"] = np.random.randint(10000,999999, size=num_providers)
    billing_df["BillingProviderTIN"] = np.random.randint(100000000,999999999, size=num_providers)
    billing_df["BillingProviderOrg"] = np.random.choice(["I", "O"], num_providers, p=[0.5, 0.5])
    billing_df['BillingProviderMName'] = [fake.first_name() for i in range(num_providers)]
    billing_df['BillingProviderFName'] = [fake.first_name() for i in range(num_providers)]
    billing_df['BillingProviderLName'] = [fake.last_name() for i in range(num_providers)]
    billing_df['BillingProviderPhoneNumber'] = [fake.msisdn() for i in range(num_providers)]
    billing_df["BillingProviderEntityType"] = np.random.choice(["I", "O"], num_providers, p=[0.5, 0.5])
    billing_df["BillingProviderNPI"]  = np.random.randint(10000,999999, size=num_providers)
    billing_df['1'] = [fake.building_number() for i in range(num_providers)]
    billing_df["2"] = [fake.street_name() for i in range(num_providers)]
    billing_df['BillingProviderAddressLine1'] = billing_df["1"]+" "+billing_df["2"]
    billing_df["1"] = np.random.choice(["Suite", "Apt.", "Null"], num_providers, p=[0.1, 0.3, 0.6])
    billing_df["2"] = [fake.building_number() for i in range(num_providers)]
    billing_df["BillingProviderAddressLine2"] = billing_df["1"]+" "+billing_df["2"]
    billing_df.loc[billing_df['BillingProviderAddressLine2'].str.contains('Null', case=False), 'BillingProviderAddressLine2'] = 'Null'
    billing_df = billing_df.drop(columns=['1',"2"])
    billing_df["BillingProviderNPI"] = billing_df["BillingProviderNPI"].astype(str)
    billing_df['BillingProviderID'] = billing_df['BillingProviderID'].astype(str)
    billing_df['BillingProviderTIN'] = billing_df['BillingProviderTIN'].astype(str)
    billing_df['BillingProviderTaxonomy'] = np.random.choice(tax[taxonomy], num_providers)
    billing_df["Billing"] = billing_df["BillingProviderID"]+"@"+billing_df["BillingProviderTIN"]+"@"+billing_df["BillingProviderOrg"]+"@"+billing_df["BillingProviderFName"]+"@"+billing_df["BillingProviderMName"]+"@"+billing_df["BillingProviderLName"]+"@"+billing_df["BillingProviderPhoneNumber"]+"@"+billing_df["BillingProviderAddressLine1"]+"@"+billing_df["BillingProviderAddressLine2"]+"@"+billing_df["BillingProviderEntityType"]+"@"+billing_df["BillingProviderNPI"]+"@"+billing_df['BillingProviderTaxonomy']
    billing_list = billing_df["Billing"].tolist()
    del billing_df
    
    group_df = pd.DataFrame()
    group_df['GroupName'] = [fake.company() for i in range(num_providers)]
    group_df['GroupNumber']  = np.random.randint(10000,999999, size=num_providers)
    group_df['GroupNumber'] = group_df['GroupNumber'].astype(str)
    group_df["Group"] = group_df['GroupName']+"@"+group_df['GroupNumber']
    group_list = group_df["Group"].tolist()
    del group_df
    
    final_df["ToothNumber"] = "Null"
    final_df["ToothBegin"] = "Null"
    final_df["ToothEnd"] = "Null"
    final_df["ToothSurfaces"] = "Null"
    
    final_df['CertifyingPhysicianName'] = " Null"
    final_df["CertifyingPhysicianID"]  = "Null"
    final_df["CertifyingPhysicianNPI"] = "Null"
    final_df["CertifyingPhysicianNPI"] ="Null"
    final_df["CertifyingPhysicianID"]  = "Null"
    
    final_df["OrderingProviderID"] = " Null"
    final_df['OrderingProviderName'] = "Null"
    final_df["OrderingProviderID"] ="Null"
    final_df["OrderingProviderNPI"] ="Null"
    
    #final_df["PrescribingProviderSpecialtyCode"]=str(taxonomy)
    final_df["PrescribingProviderSpecialtyCode"] = "Null"
    final_df["BillingProviderSpecialtyCode"]=str(taxonomy)
    final_df["RenderingProviderSpecialtyCode"]=str(taxonomy)
    
    attending_df = pd.DataFrame()
    attending_df['AttendingProviderNAME'] = [fake.last_name() for i in range(num_providers)]
    attending_df["AttendingProviderNPI"] = np.random.randint(10000,999999, size=num_providers)
    attending_df["AttendingProviderNPI"] = attending_df["AttendingProviderNPI"].astype(str)
    attending_df["Attending"] = attending_df['AttendingProviderNAME']+"@"+attending_df["AttendingProviderNPI"]
    attending_list = attending_df["Attending"].tolist()
    del attending_df
    
    """
    tooth_df = pd.DataFrame()
    tooth_df["ToothNumber"] = np.random.choice(["Null"], num_providers)
    tooth_df["ToothBegin"] = np.random.choice(["Null"], num_providers)
    tooth_df["ToothEnd"] = np.random.choice(["Null"], num_providers)
    tooth_df["ToothSurfaces"] = "Null"
    tooth_df["Tooth"] = tooth_df["ToothNumber"]+"@"+tooth_df["ToothBegin"]+"@"+tooth_df["ToothEnd"]+"@"+tooth_df["ToothSurfaces"]
    tooth_list = tooth_df["Tooth"].tolist()
    
    certifying_df = pd.DataFrame()
    change apply to astype
    certifying_df['CertifyingPhysicianNme']= [fake.last_name() for i in range(num_providers)]
    certifying_df["CertifyingPhysicianID"] = np.random.randint(10000,999999, size=num_providers)
    certifying_df["CertifyingPhysicianNPI"] = np.random.randint(10000,999999, size=num_providers)
    certifying_df["CertifyingPhysicianNPI"] =certifying_df["CertifyingPhysicianNPI"].apply(str)
    certifying_df["CertifyingPhysicianID"] = certifying_df["CertifyingPhysicianID"].apply(str)
    certifying_df["Certifying"] =certifying_df["CertifyingPhysicianNme"]+"@"+certifying_df['CertifyingPhysicianID']+"@"+certifying_df["CertifyingPhysicianNPI"]
    certifying_list = certifying_df["Certifying"].tolist()
    
    ordering_df = pd.DataFrame()
    ordering_df["OrderingProviderID"] = np.random.randint(10000,999999, size=num_providers)
    ordering_df['OrderingProviderName'] = [fake.last_name() for i in range(num_providers)]
    ordering_df["OrderingProviderID"] =ordering_df["OrderingProviderID"].astype(str)
    ordering_df["OrderingProviderNPI"]  = np.random.randint(10000,999999, size=num_providers)
    ordering_df["OrderingProviderNPI"] = ordering_df["OrderingProviderNPI"].astype(str)
    ordering_df["Ordering"] =ordering_df["OrderingProviderID"]+"@"+ordering_df['OrderingProviderName']+"@"+ordering_df["OrderingProviderNPI"]
    ordering_list = ordering_df["Ordering"].tolist()
    
    prescribing_df = pd.DataFrame()
    prescribing_df["PrescribingProviderID"] = np.random.randint(10000,999999, size=num_providers)
    prescribing_df["PrescribingProviderTIN"] = np.random.randint(100000000,999999999, size=num_providers)
    prescribing_df["PrescribingProviderOrg"] = np.random.choice(["I", "O"], num_providers, p=[0.5, 0.5])
    prescribing_df['PrescribingProviderFName'] = [fake.first_name() for i in range(num_providers)]
    prescribing_df['PrescribingProviderMName'] = [fake.first_name() for i in range(num_providers)]
    prescribing_df['PrescribingProviderLName'] = [fake.last_name() for i in range(num_providers)]
    prescribing_df['PrescribingProviderPhoneNumber'] = [fake.msisdn() for i in range(num_providers)]
    prescribing_df['1'] = [fake.building_number() for i in range(num_providers)]
    prescribing_df["2"] = [fake.street_name() for i in range(num_providers)]
    prescribing_df['PrescribingProviderAddressLine1'] = prescribing_df["1"]+" "+prescribing_df["2"]
    prescribing_df["1"] = np.random.choice(["Suite", "Apt.", "Null"], num_providers, p=[0.1, 0.3, 0.6])
    prescribing_df["2"] = [fake.building_number() for i in range(num_providers)]
    prescribing_df["PrescribingProviderAddressLine2"] = prescribing_df["1"]+" "+prescribing_df["2"]
    prescribing_df.loc[prescribing_df['PrescribingProviderAddressLine2'].str.contains('Null', case=False), 'PrescribingProviderAddressLine2'] = 'Null'
    prescribing_df['PrescribingProviderCity'] = [fake.city() for i in range(num_providers)]
    prescribing_df = prescribing_df.drop(columns=['1',"2"])
    prescribing_df["PrescribingProviderNPI"] = np.random.randint(10000,999999, size=num_providers)
    prescribing_df['PrescribingProviderNPI'] = prescribing_df['PrescribingProviderNPI'].apply(str)
    prescribing_df['PrescribingProviderID'] = prescribing_df['PrescribingProviderID'].apply(str)
    prescribing_df['PrescribingProviderTIN'] = prescribing_df['PrescribingProviderTIN'].apply(str)
    prescribing_df["Prescribing"] = prescribing_df["PrescribingProviderID"]+"@"+prescribing_df["PrescribingProviderTIN"]+"@"+prescribing_df["PrescribingProviderOrg"]+"@"+ prescribing_df['PrescribingProviderFName']+"@"+prescribing_df['PrescribingProviderMName']+"@"+prescribing_df['PrescribingProviderLName']+"@"+prescribing_df['PrescribingProviderPhoneNumber'] +"@"+ prescribing_df['PrescribingProviderAddressLine1']+"@"+prescribing_df["PrescribingProviderAddressLine2"]+"@"+prescribing_df['PrescribingProviderCity']+"@"+prescribing_df['PrescribingProviderNPI']
    prescribing_list = prescribing_df["Prescribing"].tolist()"""
    
    final_df["PrescribingProviderID"] = "Null"
    final_df["PrescribingProviderTIN"] = "Null"
    final_df["PrescribingProviderOrg"] = "Null"
    final_df['PrescribingProviderFName'] = "Null"
    final_df['PrescribingProviderMName'] = "Null"
    final_df['PrescribingProviderLName'] = "Null"
    final_df['PrescribingProviderPhoneNumber'] = "Null"
    final_df['PrescribingProviderAddressLine1'] = "Null"
    final_df["PrescribingProviderAddressLine2"] = "Null"
    final_df['PrescribingProviderCity'] = "Null"
    final_df["PrescribingProviderNPI"] = "Null"
    final_df['PrescribingProviderNPI'] = "Null"
    final_df['PrescribingProviderID'] = "Null"
    final_df['PrescribingProviderTIN'] = "Null"
    final_df["PrescribingProviderTaxonomy"]="Null"
    final_df["ClaimType"] = "P" #P for profesional, D for dental, F for facility
    
    final_df["ContractedFlag"] = np.random.choice(["Y", "N"], num_lines, p=[0.5, 0.5])
    final_df["InterPlan"] = np.random.choice(["Y", "N"], num_lines, p=[0.5, 0.5])
    final_df["ASOFlag"] = np.random.choice(["Y", "N"], num_lines, p=[0.5, 0.5])
    final_df["PrimaryFlag"] = np.random.choice(["Y", "N"], num_lines, p=[0.5, 0.5])
    final_df["OutOfNetwork"] = np.random.choice(["Y", "N"], num_lines, p=[0.5, 0.5])
    final_df["CapEpcIndicator"] = np.random.choice(["Y", "N"], num_lines, p=[0.5, 0.5])
    final_df["DenyFlag"] = np.random.choice(["Y", "N"], num_lines, p=[0.1, 0.9])
    final_df["PreAuthIndicator"] = np.random.choice(["Y", "N"], num_lines, p=[0.5, 0.5])
    final_df["DenyFlag"] = np.random.choice(["P", "D"], num_lines, p=[0.5, 0.5])
    final_df["PayeeType"] = np.random.choice(["Prov", "Subscriber"], num_lines, p=[0.9, 0.1])
    final_df["LineConsideredAmount"] =  "Null"
    final_df["PreAuthNumber"] = "Null"
    final_df["ICDVersion"] = np.random.choice(["ICD-10","ICD-9"], num_lines, p =[0.8,0.2])
    final_df["PriorClaimID"] = "Null"
    final_df["LineAdjustmentReasonCode"] = "Null"
    final_df["RelatedHospDischargeDate"] = "Null"
    final_df["RelatedHospAdmitDate"] = "Null"
    final_df["ClaimAdjustmentReasonCode"] = "Null"
    final_df["NDC"] = "Null"
    final_df["LineAllowedAmount"] = "Null"
    final_df["DenyReason"] = "Null"
    final_df["ConditionCode"] = "Null"
    final_df["ValueCode"] = "Null"#only not "Null" for facility claims
    final_df["ServiceUnits"] = "Null"
    final_df["LineUnits"] = "Null"
    final_df["LineUnitType"] = "Null"
    final_df["CLIA"]="NUll"# null for all but Clinical labs
    
    final_df["LineOfBusiness"] = np.random.choice(["Commercial", "Medicare", "Commercial Plus"], num_lines, p=[0.3, 0.4, 0.3])
    final_df["InterPlan"] = np.random.choice(["ITS", "nonITS", "HOST"], num_lines, p=[0.3, 0.4, 0.3])
    final_df["OccurrenceCode"] = np.random.choice(["00", "01","02","03","04","05","06","09",], num_lines, p=[0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125])
    final_df["DRGVersion"] = np.random.choice(["37", "38","39"], num_lines, p=[0.4, 0.3,0.3])
    final_df["RelatedCausesCode"] = np.random.choice(["AA", "AP","EM", "OA"], num_lines, p=[0.2, 0.1,0.3, 0.4])
    
    final_df["ReferringProviderNPI"] = np.random.randint(10000,999999, size=num_lines)
    final_df["OtherPhysicianNPI"] = np.random.randint(10000,999999, size=num_lines)
    final_df["AdmitSource"]= np.random.randint(1, 9, size=num_lines)
    final_df["AdmitType"] =np.random.randint(1, 9, size=num_lines)
    final_df["DischargeStatus"] = np.random.randint(0, 99, size=num_lines)
    final_df['OrderingProviderName'] = [fake.last_name() for i in range(num_lines)]
    final_df['OtherPhysicianNAME'] = [fake.last_name() for i in range(num_lines)]
    
    final_df["TypeofBill"]= np.random.randint(0,999, size=num_lines)
    final_df["TypeofBill"]=final_df["TypeofBill"].apply(lambda x: '{0:0>3}'.format(x))
    final_df["NumofAdjustments"]= np.random.randint(0,10, size=num_lines)
    final_df["RevenueCode"]= np.random.randint(0,9999, size=num_lines)
    final_df["RevenueCode"]=final_df["RevenueCode"].apply(lambda x: '{0:0>4}'.format(x))
    final_df["TypeofBill"]= np.random.randint(0,999, size=num_lines)
    final_df["TypeofBill"]=final_df["TypeofBill"].apply(lambda x: '{0:0>4}'.format(x))
    
    final_df["Billing"] = np.random.choice(billing_list, size=num_lines)
    final_df["Group"] = np.random.choice(group_list, size=num_lines)
    
    #final_df["Tooth"] = np.random.choice(tooth_list, size=num_lines)
    #final_df["Prescribing"] = np.random.choice(prescribing_list, size=num_lines)
    #final_df["Ordering"] = np.random.choice(ordering_list, size=num_lines)
    #final_df["Certifying"] = np.random.choice(certifying_list, size=num_lines) 
    
    adult_df = final_df.loc[(final_df['age_bin'] == 'Adult')]
    adult_df["Pat"] = np.random.choice(adult_list, size=adult_df.shape[0])
    
    elderly_df = final_df.loc[(final_df['age_bin'] == 'Elderly')]
    elderly_df["Pat"] = np.random.choice(elderly_list, size=elderly_df.shape[0])
    
    minor_df = final_df.loc[(final_df['age_bin'] == 'Minor')]
    minor_df["Pat"] = np.random.choice(minor_list, size=minor_df.shape[0])
    
    final_df = pd.concat([minor_df, elderly_df, adult_df])
    del adult_df
    del elderly_df
    del minor_df
    
    final_df["Attending"] = np.random.choice(attending_list, size=num_lines)
    final_df["AccidentDate"] = np.random.choice(pd.date_range('2020-01-01', '2022-01-01'), len(final_df))
    delta = pd.to_timedelta(np.random.randint(1,10, size=len(final_df)), unit='d')
    final_df["AdmitDate"] = final_df["AccidentDate"]+delta
    delta = pd.to_timedelta(np.random.randint(1,10, size=len(final_df)), unit='d')
    final_df["ClaimServiceStartDate"] = final_df["AdmitDate"] +delta
    delta = pd.to_timedelta(np.random.randint(1,10, size=len(final_df)), unit='d')
    final_df["LineServiceStartDate"] = final_df["ClaimServiceStartDate"]+delta
    delta = pd.to_timedelta(np.random.randint(1,10, size=len(final_df)), unit='d')
    final_df["LineServiceEndDate"]=final_df["LineServiceStartDate"]  + delta
    delta = pd.to_timedelta(np.random.randint(1,10, size=len(final_df)), unit='d')
    final_df["ClaimServiceEndDate"]=final_df["LineServiceEndDate"]+delta 
    delta = pd.to_timedelta(np.random.randint(1,10, size=len(final_df)), unit='d')
    final_df["ClaimReceivedDate"] = final_df["ClaimServiceEndDate"]+delta
    delta = pd.to_timedelta(np.random.randint(1,10, size=len(final_df)), unit='d')
    final_df["ClaimPaidDate"] = final_df["ClaimReceivedDate"] +delta
    final_df["LineServiceStatusCategoryCode"] = final_df["AdmitDate"]
    
    final_df["EmergencyIndicator"] = "Null"
    final_df["AdmittingDiag"]= "Null"
    final_df["ICDProcDate1"]="Null"
    final_df["ICDProcDate2"]="Null"
    final_df["ICDProcDate3"]="Null"
    final_df["ICDProcDate4"]="Null"
    final_df["ICDProc1"]= "Null"
    final_df["ICDProc2"]= "Null"
    final_df["ICDProc3"]= "Null"
    final_df["ICDProc4"]= "Null"
    
    final_df["HIPPSCode"]= "Null"
    final_df["ClaimStatusCategoryCode"] = "Null"
    final_df["HCPC_Rate_HIPPSCode"]= "Null"
    final_df["DRGCode"]= "Null"
    
    """final_df["ICDProcDate1"]=final_df["LineServiceStartDate"]     #for facility claims
    final_df["ICDProcDate2"]=final_df["LineServiceStartDate"]
    final_df["ICDProcDate3"]=final_df["LineServiceStartDate"]
    final_df["ICDProcDate4"]=final_df["LineServiceStartDate"]"""
    
    
    def splitting(name, delineator = "@"):
        final_df[name] = final_df[name].apply(lambda x: x.split(delineator))
    
    splitting("Billing")
    splitting("Group")
    splitting("Attending")
    splitting("Pat")
    
    #splitting("Prescribing")
    #splitting("Ordering")
    #splitting("Certifying")
    #splitting("Tooth")
    
    final_df[["BillingProviderID", "BillingProviderTIN", "BillingProviderOrg", "BillingProviderFName", "BillingProviderMName", "BillingProviderLName", "BillingProviderPhoneNumber", "BillingProviderAddressLine1", "BillingProviderAddressLine2","BillingProviderEntityType","BillingProviderNPI",'BillingProviderTaxonomy']] = pd.DataFrame(final_df.pop('Billing').values.tolist(), index=final_df.index)
    final_df[["GroupName","GroupNumber"]] = pd.DataFrame(final_df.pop('Group').values.tolist(), index=final_df.index)
    final_df[["AttendingProviderNAME","AttendingProviderNPI"]] = pd.DataFrame(final_df.pop('Attending').values.tolist(), index=final_df.index)
    final_df[['PatientState', 'PatientFName', 'pat_gender', 'PatientLName', 'PatientMName', 'PatientID', 'PatientAddressLine1', 'PatientAddressLine2', 'SubscriberState', 'SubscriberID', 'PatientRelationship', 'PatientDOB', 'SubscriberGender', 'SubscriberDOB', 'SubscriberAddressLine1', 'SubscriberAddressLine2', 'SubscriberLName', 'SubscriberFName', 'SubscriberMName']] = pd.DataFrame(final_df.pop('Pat').values.tolist(), index=final_df.index)
    final_df[rend] = pd.DataFrame(final_df.pop('Ren').values.tolist(), index=final_df.index)
    
    
    #final_df[["PrescribingProviderID", "PrescribingProviderTIN", "PrescribingProviderOrg", 'PrescribingProviderFName', 'PrescribingProviderMName', 'PrescribingProviderLName', 'PrescribingProviderPhoneNumber', "PrescribingProviderAddressLine1", "PrescribingProviderAddressLine2","PrescribingProviderNPI"]] = pd.DataFrame(final_df.pop('Prescribing').values.tolist(), index=final_df.index)
    #final_df[["OrderingProviderID", 'OrderingProviderName',"OrderingProviderNPI"]] = pd.DataFrame(final_df.pop('Ordering').values.tolist(), index=final_df.index)
    #final_df[['CertifyingPhysicianName', "CertifyingPhysicianID", "CertifyingPhysicianNPI"]] = pd.DataFrame(final_df.pop('Certifying').values.tolist(), index=final_df.index)
    #final_df[["ToothNumber","ToothBegin", "ToothEnd", "ToothSurfaces"]] = pd.DataFrame(final_df.pop('Tooth').values.tolist(), index=final_df.index)
    
    combo_list = final_df["combos"].tolist()
    
    na_df = final_df.loc[(final_df['PickUpZipCode'] == '0')]
    amb_df = final_df.loc[(final_df['PickUpZipCode'] != '0')]
    na_df["APCCode"] = "Null"
    amb_df["APCCode"] =np.random.randint(10000,99999, size=amb_df.shape[0])
    
    final_df = pd.concat([amb_df, na_df])
    del amb_df
    del na_df
    
    line_df = pd.DataFrame()
    line_df["len"] = final_df['combos'].str.len()
    line_df['LineNumber'] = line_df.apply(lambda x: list(range(1, x.len + 1)), axis=1)
    line_df= line_df.explode('LineNumber')
    
    
    final_df = final_df.explode('combos')#spliting up line into HCPCS and mod value in list, first is Hcpcs, second is mod for line
    final_df["combos"] = final_df["combos"].apply(lambda x: x.split(";"))
    final_df[['hcpcs_code','mod_values']] = pd.DataFrame(final_df.pop('combos').values.tolist(), index=final_df.index)
    
    final_df['ren_id'] = final_df['ren_id'].astype(str)
    rendering_df['ren_id'] = rendering_df['ren_id'].astype(str)
    final_df["combo"] = final_df["hcpcs_code"] +"@"+final_df["mod_values"]
    
    # cost = pd.read_csv(line_amt_csv, dtype={'place_of_service': str}, encoding= 'unicode_escape')
    # cost['hcpcs_code'] = cost['hcpcs_code'].astype(str)
    # cost['mod_values'] = cost['mod_values'].astype(str)
    # 
    # 
    # cost['combo'] = cost.apply(lambda x: x.hcpcs_code.split('@') + x.mod_values.split('@'), axis=1)
    # cost['combo'] = cost['combo'].apply(sorted)
    # cost['combo'] = cost['combo'].transform(lambda x: '@'.join(x))
    # 
    # cost = cost.drop(columns=['diag_values', 'place_of_service',"pat_gender", "mod_values", "proc_values"])
    # 
    # 
    # c = cost.groupby(['combo'])['amount_charged'].sum()
    # c = c.reset_index()
    # e = cost.groupby(['combo'],as_index = False).size()
    # c["count"] = e["size"]
    # c["avg_charge"] = c["amount_charged"]/ c["count"]
    # 
    # ds = cost[cost.amount_paid !=0]
    # d = ds.groupby(['combo'])['amount_paid'].sum()
    # d = d.reset_index()
    # e = ds.groupby(['combo'], as_index= False).size()
    # d["count"] = e["size"]
    # d = d.merge(c[["combo", "avg_charge"]])
    # d["avg_paid_pct"] = (d["amount_paid"]/ d["count"])/d["avg_charge"]
    # 
    # f = cost.groupby(['combo'])['amount_paid'].sum()
    # f = f.reset_index()
    # e = cost.groupby(['combo'], as_index=False).size()
    # 
    # f["count"] = e["size"]
    # f = f.merge(c[["combo", "avg_charge"]])
    # f["avg_paid_pct"] = (f["amount_paid"]/ f["count"])/f["avg_charge"]
    # f = f[f.avg_paid_pct == 0.000000]
    # 
    # d = d.append(f)
    
    c = pd.read_csv(c_amt_csv)
    d = pd.read_csv(d_amt_csv)
    
    scb = 50      #may want to change scale so that it is more in line with reality
    sca = .05
    
    cost_dict = c.set_index('combo').to_dict()
    final_df['cost'] = final_df['combo'].apply(lambda x: cost_dict["avg_charge"][x])
    final_df['real_charge'] = np.random.normal(final_df['cost'], scale = scb).astype(int)
    final_df['real_charge'] = final_df['real_charge'].abs()
    final_df = final_df.drop(columns=['cost'])
    
    paid_dict = d.set_index('combo').to_dict()
    final_df['paid'] = final_df['combo'].apply(lambda x: paid_dict["avg_paid_pct"][x])
    final_df['real paid'] = np.random.normal(final_df['paid'], scale = sca)*final_df["real_charge"].astype(int)
    final_df['real paid'] = final_df['real paid'].abs()
    final_df = final_df.drop(columns=['paid'])
    final_df["real paid"] = final_df["real paid"].astype(int)
    
    
    final_df["claim"] = final_df["prov_claim_id"]+"@"+final_df["ren_id"]
    
    final_df_1 = final_df
    final_df_1.groupby(['claim']).agg({'real_charge': sum, 'real paid': sum}).reset_index()
    
    final_df["total charged"] = final_df_1["real_charge"]
    final_df["total paid"] = final_df_1["real paid"]
    final_df = final_df.drop(columns=['claim'])
    
    final_df['Specialty_code'] = "25"
    
    final_df["LineNumber"]=line_df["LineNumber"]
    del line_df
    
    def splitting3(col_name, num):
        col_name = col_name.split('@')
        null_vals = num - len(col_name)
        null_lst = ["Null"] * null_vals
        col_name = col_name + null_lst
        return col_name
    final_df['mod_values'] = final_df['mod_values'].apply(lambda x: splitting3(x, 6))
    final_df['diag_values'] = final_df['diag_values'].apply(lambda x: splitting3(x, 13))
    
    
    final_df[["MOD1","MOD2","MOD3","MOD4","MOD5","MOD6"]] = pd.DataFrame(final_df.pop('mod_values').values.tolist(),
                                                                         index=final_df.index)
    final_df[["Diag1", "Diag2", "Diag3", "Diag4", "Diag5", "Diag6",
              "Diag7", "Diag8", "Diag9", "Diag10", "Diag11",
              "Diag12","Diag13"]] = pd.DataFrame(final_df.pop('diag_values').values.tolist(), index=final_df.index)
    
    final_df.rename(columns = {'pat_gender':'PatientGender', "place_of_service": "LinePOS",
                               "total paid":"LineTotPaid", "total charged":"LineCharges",
                               "hcpcs_code":"LineProcCode"}, inplace = True)
    final_df = final_df.drop(columns=['combo_tot', 'Unnamed: 0', "age_bin", 'combo',
                                      "real_charge", "real paid", 'ren_id', "Specialty_code",
                                      "prov_claim_id"])
    
    
    final_df["LineTotPaid"] = final_df["LineTotPaid"].astype(int) 
    nm_df = final_df.loc[(final_df['LineTotPaid'] <= 20)]
    sm_df = final_df.loc[(final_df['LineTotPaid'] >= 20)]
    nm_df["LineAmountCopay"] = nm_df["LineTotPaid"]
    
    nm_df["LineAmountCoinsurance"] = 0
    nm_df["LineAmountDeductible"] = 0
    nm_df["LinePayerPaid"] = 0
    sm_df["LineAmountCoinsurance"] = np.random.randint(0,20, size=sm_df.shape[0])/100
    sm_df["LineAmountDeductible"]= np.random.randint(0,40, size=sm_df.shape[0])/100
    
    sm_df["LineAmountCopay"]=[random.randrange(5,21,5)for i in range(sm_df.shape[0])]
    sm_df["LinePayerPaid"]=sm_df["LineTotPaid"]-sm_df["LineAmountCopay"]
    
    sm_df["LineAmountCoinsurance"] = sm_df["LineAmountCoinsurance"] * sm_df["LinePayerPaid"]//1
    sm_df["LinePayerPaid"]=sm_df["LineTotPaid"]-sm_df["LineAmountCopay"]- sm_df["LineAmountCoinsurance"] 
    sm_df["LineAmountDeductible"] = sm_df["LineAmountDeductible"] * sm_df["LinePayerPaid"]//1
    
    final_df = pd.concat([sm_df, nm_df])
    del sm_df
    del nm_df
    final_df["LinePatientPaid"] = final_df["LineAmountCopay"] + final_df["LineAmountDeductible"
                                                                        ] + final_df["LineAmountCoinsurance"]
    
    final_df["LineCOBPay"] = "0"
    final_df["HdrCOBPay"] = "0"
    final_df = final_df.sort_index(ascending=True)
    
    hdr_df= final_df[["ClaimID","LineTotPaid", "LineAmountCopay", "LinePayerPaid",  "LineCharges","LinePatientPaid" ]]
    
    
    hdr_df = hdr_df.groupby(['ClaimID'], as_index=False).agg({'LineTotPaid': 'sum',
                                                              'LinePatientPaid': 'sum',
                                                              'LineAmountCopay': 'sum',
                                                              'LinePayerPaid': 'sum',
                                                              'LineCharges': 'sum'})
    hdr_df.rename(columns={'LineTotPaid':'HdrPay',
                           'LinePayerPaid': 'HdrPayerPay',
                           'LineAmountCopay': 'HdrPatientCopay',
                           'LineCharges': 'HdrCharges',
                           'LinePatientPaid': 'HdrPatientPay'}, inplace=True)
    
    final_df = pd.merge(final_df, hdr_df, on='ClaimID')
    del hdr_df
    
    final_df = final_df[["ClaimID", "CapEpcIndicator", "SubscriberID", "SubscriberFName",
     "SubscriberMName", "SubscriberLName", "SubscriberDOB", "SubscriberGender", "SubscriberAddressLine1",
     "SubscriberAddressLine2", "SubscriberCity", "SubscriberState", "SubscriberZipCode", "PatientID",
     "PatientRelationship", "PatientFName", "PatientMName", "PatientLName", "PatientDOB", "PatientGender",
     "PatientAddressLine1", "PatientAddressLine2", "PatientCity", "PatientState", "PatientZipCode",
     "GroupNumber", "GroupName", "BillingProviderID", "BillingProviderNPI", "BillingProviderTIN",
     "BillingProviderSpecialtyCode", "BillingProviderTaxonomy", "BillingProviderOrg", "BillingProviderFName",
     "BillingProviderMName", "BillingProviderLName", "BillingProviderEntityType", "BillingProviderAddressLine1",
     "BillingProviderAddressLine2", "BillingProviderCity", "BillingProviderState", "BillingProviderZipCode",
     "BillingProviderPhoneNumber", "CertifyingPhysicianID", "CertifyingPhysicianName", "CertifyingPhysicianNPI",
     "OrderingProviderID", "OrderingProviderName", "OrderingProviderNPI", "ReferringProviderNPI", "InterPlan",
     "LineOfBusiness", "ClaimPaidDate", "ClaimReceivedDate", "ClaimServiceEndDate", "ClaimServiceStartDate",
     "ClaimAdjustmentReasonCode", "PriorClaimID", "ASOFlag", "PrimaryFlag", "ClaimStatusCategoryCode",
     "ClaimType", "ContractedFlag", "HdrCharges", "HdrCOBPay", "HdrPatientCopay", "HdrPatientPay", "HdrPay",
     "HdrPayerPay", "CLIA", "DropOffZipCode", "PickUpZipCode", "RelatedCausesCode", "AccidentDate",
     "EmergencyIndicator", "LineNumber", "NumofAdjustments", "LinePatientPaid", "LineAmountCoinsurance",
     "LineAmountCopay", "LineAmountDeductible", "LineCharges", "LinePayerPaid", "LineCOBPay", "LineTotPaid",
     "LineProcCode", "LineUnits", "LineUnitType", "LineConsideredAmount", "LineAllowedAmount", "MOD1", "MOD2",
     "MOD3", "MOD4", "MOD5", "MOD6", "Diag1", "Diag2", "Diag3", "Diag4", "Diag5", "Diag6", "Diag7", "Diag8",
     "Diag9", "Diag10", "Diag11", "Diag12", "Diag13", "ICDVersion", "LinePOS", "LineServiceEndDate",
     "LineServiceStartDate", "LineServiceStatusCategoryCode", "LineAdjustmentReasonCode", "RenderingProviderID",
     "RenderingProviderNPI", "RenderingProviderTIN", "RenderingProviderSpecialtyCode", "RenderingProviderTaxonomy",
     "RenderingProviderOrg", "RenderingProviderFName", "RenderingProviderMName", "RenderingProviderLName",
     "RenderingProviderEntityType", "RenderingProviderAddressLine1", "RenderingProviderAddressLine2",
     "RenderingProviderCity", "RenderingProviderState", "RenderingProviderZipCode", "RenderingProviderPhoneNumber",
     "RelatedHospAdmitDate", "RelatedHospDischargeDate", "AdmitType", "AdmitSource", "DischargeStatus", "DRGCode",
     "DRGVersion", "AdmittingDiag", "ICDProc1", "ICDProc2", "ICDProc3", "ICDProc4", "ICDProcDate1", "ICDProcDate2",
     "ICDProcDate3", "ICDProcDate4", "APCCode", "CCN", "AttendingProviderNAME", "AttendingProviderNPI",
     "OtherPhysicianNAME", "OtherPhysicianNPI", "RevenueCode", "TypeofBill", "HCPC_Rate_HIPPSCode", "OccurrenceCode",
     "ServiceUnits", "PrescribingProviderID", "PrescribingProviderNPI", "PrescribingProviderTIN",
     "PrescribingProviderSpecialtyCode", "PrescribingProviderTaxonomy", "PrescribingProviderOrg",
     "PrescribingProviderFName", "PrescribingProviderMName", "PrescribingProviderLName",
     "PrescribingProviderAddressLine1", "PrescribingProviderAddressLine2", "PrescribingProviderCity",
     "PrescribingProviderState", "PrescribingProviderZipCode", "PrescribingProviderPhoneNumber", "OutOfNetwork",
     "PreAuthIndicator", "PreAuthNumber", "NDC", "DenyFlag", "DenyReason", "HIPPSCode", "AdmitDate", "ToothNumber",
     "ToothBegin", "ToothEnd", "ToothSurfaces", "ValueCode", "ConditionCode", "PayeeType"]]
    
    
    
    """
    final_df['icdproc5'] = 'Null'
    final_df['otherproviderzipcode'] = 'Null'
    final_df['serviceproviderorg'] = 'Null'
    final_df['certifyingprovideraddressline1'] = 'Null'
    final_df['certifyingprovideraddressline2'] = 'Null'
    final_df['flex_fld_1'] = 'Null'
    final_df['serviceproviderentitytype'] = 'Null'
    final_df['serviceproviderspecialtycode2'] = 'Null'
    final_df['otherprovidertin'] = 'Null'
    final_df['etl_billingprovider_addr_key'] = 'Null' 
    final_df['etl_serviceprovider_key'] = 'Null' 
    final_df['icdproc28date'] = 'Null' 
    final_df['orderingproviderorg'] = 'Null'
    final_df['referringprovidertaxonomy'] = 'Null'
    final_df['renderingprovideraveragelinecharge'] = 'Null'
    final_df['etl_certifyingprovidershist_key'] = 'Null'
    final_df['etl_serviceprovider_addr_key'] = 'Null'
    final_df['otherproviderphonenumber'] = 'Null' 
    final_df['etl_orderingprovider_addr_key'] = 'Null' 
    final_df['icdproc14'] = 'Null' 
    final_df['cdx_aux_4'] = 'Null' 
    final_df['attendingprovideraveragelinecharge'] = 'Null' 
    final_df['operatingproviderentitytype'] = 'Null' 
    final_df['renderingproviderspecialtycode1'] = 'Null'
    final_df['etl_supervisingprovidershist_key'] = 'Null'
    final_df['serviceprovideraddressline1'] = 'Null' 
    final_df['cdx_aux_date_1'] = 'Null' 
    final_df['otherprovidertaxonomy'] = 'Null'
    final_df['etl_orderingprovider_key'] = 'Null'
    final_df['otherprovidermname'] = 'Null' 
    final_df['icdproc18date'] = 'Null'
    final_df['supervisingproviderlname'] = 'Null' 
    final_df['icdproc11date'] = 'Null'
    final_df['cdx_aux_3'] = 'Null' 
    final_df['attendingproviderspecialtycode3'] = 'Null'
    final_df['condition_code'] = 'Null' 
    final_df['icdproc1date'] = 'Null' 
    final_df['icdproc17'] = 'Null'
    final_df['hcpcratehippscode'] = 'Null'
    final_df['flex_fld_date_3'] = 'Null'
    final_df['icdproc7date'] = 'Null' 
    final_df['referringproviderentitytype'] = 'Null'
    final_df['etl_otherprovider_key'] = 'Null'
    final_df['operatingproviderid'] = 'Null'
    final_df['certifyingproviderlname'] = 'Null'
    final_df['etl_operatingprovider_addr_key'] = 'Null'
    final_df['cdx_aux_date_2'] = 'Null' 
    final_df['icdproc13date'] = 'Null'
    final_df['referringproviderid'] = 'Null'
    final_df['operatingproviderlname'] = 'Null'
    final_df['cdx_aux_num_4'] = 'Null' 
    final_df['etl_attendingprovider_key'] = 'Null'
    final_df['cdx_aux_date_3'] = 'Null'
    final_df['supervisingprovidertaxonomy'] = 'Null' 
    final_df['icdproc20'] = 'Null' 
    final_df['supervisingproviderentitytype'] = 'Null' 
    final_df['attendingproviderstate'] = 'Null' 
    final_df['etl_orderingprovidershist_key'] = 'Null'
    final_df['icdproc28'] = 'Null' 
    final_df['certifyingproviderentitytype'] = 'Null' 
    final_df['icdproc27date'] = 'Null'
    final_df['diag26'] = 'Null' 
    final_df['certifyingproviderspecialtycode3'] = 'Null' 
    final_df['certifyingprovidermname'] = 'Null' 
    final_df['cdx_aux_5'] = 'Null'
    final_df['etl_subscriber_key'] = 'Null' 
    final_df['diag18'] = 'Null' 
    final_df['icdproc15'] = 'Null'
    final_df['otherproviderstate'] = 'Null' 
    final_df['etl_renderingprovider_addr_key'] = 'Null' 
    final_df['attendingprovidermname'] = 'Null'
    final_df['attendingproviderlname'] = 'Null' 
    final_df['operatingprovidernpi'] = 'Null' 
    final_df['etl_patientbeneshist_key'] = 'Null' 
    final_df['certifyingproviderorg'] = 'Null' 
    final_df['etl_renderingprovidershist_key'] = 'Null' 
    final_df['referringproviderstate'] = 'Null'
    final_df['occurrencecode2'] = 'Null'
    final_df['icdproc17date'] = 'Null' 
    final_df['orderingprovideraddressline1'] = 'Null' 
    final_df['attendingprovideraddressline1'] = 'Null' 
    final_df['orderingproviderstate'] = 'Null' 
    final_df['icdproc20date'] = 'Null'
    final_df['supervisingproviderspecialtycode3'] = 'Null'
    final_df['icdproc19date'] = 'Null' 
    final_df['supervisingproviderspecialtycode1'] = 'Null'
    final_df['prescribingproviderentitytype'] = 'Null' 
    final_df['attendingprovidertaxonomy'] = 'Null'
    final_df['icdproc5date'] = 'Null'
    final_df['certifyingprovidercity'] = 'Null'
    final_df['referringproviderphonenumber'] = 'Null'
    final_df['operatingproviderspecialtycode2'] = 'Null'
    final_df['etl_operatingprovidershist_key'] = 'Null' 
    final_df['referringprovidermname'] = 'Null' 
    final_df['icdproc10date'] = 'Null' 
    final_df['diag27'] = 'Null' 
    final_df['icdproc19'] = 'Null' 
    final_df['diag16'] = 'Null' 
    final_df['operatingprovidertaxonomy'] = 'Null' 
    final_df['operatingproviderzipcode'] = 'Null' 
    final_df['operatingprovidercity'] = 'Null' 
    final_df['icdproc9date'] = 'Null'
    final_df['operatingproviderspecialtycode1'] = 'Null'
    final_df['icdproc26date'] = 'Null'
    final_df['icdproc22date'] = 'Null' 
    final_df['orderingproviderentitytype'] = 'Null'
    final_df['diag23'] = 'Null'
    final_df['occurrencecode8'] = 'Null'
    final_df['icdproc7'] = 'Null'
    final_df['operatingprovidermname'] = 'Null'
    final_df['serviceprovidertaxonomy'] = 'Null'
    final_df['flex_fld_2'] = 'Null'
    final_df['supervisingproviderstate'] = 'Null'
    final_df['operatingproviderspecialtycode3'] = 'Null'
    final_df['icdproc4date'] = 'Null'
    final_df['supervisingprovidermname'] = 'Null'
    final_df['certifyingproviderzipcode'] = 'Null'
    final_df['certifyingproviderphonenumber'] = 'Null'
    final_df['etl_otherprovidershist_key'] = 'Null'
    final_df['referringproviderzipcode'] = 'Null'
    final_df['orderingproviderzipcode'] = 'Null'
    final_df['etl_patient_key'] = 'Null'
    final_df['otherproviderfname'] = 'Null'
    final_df['cdx_aux_num_2'] = 'Null'
    final_df['otherprovidercity'] = 'Null'
    final_df['etl_attendingprovider_addr_key'] = 'Null'
    final_df['etl_patient_addr_key'] = 'Null'
    final_df['supervisingproviderphonenumber'] = 'Null'
    final_df['cdx_aux_1'] = 'Null' 
    final_df['lineamountconsidered'] = 'Null'
    final_df['diag20'] = 'Null'
    final_df['custpatientid'] = 'Null'
    final_df['flex_fld_date_4'] = 'Null'
    final_df['icdproc29'] = 'Null'
    final_df['serviceproviderfname'] = 'Null'
    final_df['etl_prescribingprovidershist_key'] = 'Null'
    final_df['renderingproviderspecialtycode3'] = 'Null' 
    final_df['etl_billingprovider_key'] = 'Null' 
    final_df['flex_fld_date_1'] = 'Null' 
    final_df['certifyingproviderspecialtycode2'] = 'Null'
    final_df['otherprovideraddressline1'] = 'Null'
    final_df['prescribingproviderspecialtycode1'] = 'Null'
    final_df['supervisingproviderzipcode'] = 'Null' 
    final_df['serviceprovideraddressline2'] = 'Null'
    final_df['diag21'] = 'Null'
    final_df['orderingprovidercity'] = 'Null' 
    final_df['referringproviderorg'] = 'Null' 
    final_df['icdproc12'] = 'Null' 
    final_df['certifyingprovidernpi'] = 'Null'
    final_df['etl_renderingprovider_key'] = 'Null' 
    final_df['icdproc24'] = 'Null'
    final_df['supervisingproviderid'] = 'Null'
    final_df['contract_percentage'] = 'Null' 
    final_df['referringproviderspecialtycode3'] = 'Null'
    final_df['occurrencecode7'] = 'Null' 
    final_df['icdproc11'] = 'Null' 
    final_df['icdproc14date'] = 'Null'
    final_df['serviceprovidertin'] = 'Null'
    final_df['claim_service_type'] = 'Null'
    final_df['referringproviderlname'] = 'Null'
    final_df['icdproc22'] = 'Null' 
    final_df['etl_supervisingprovider_addr_key'] = 'Null' 
    final_df['orderingproviderspecialtycode3'] = 'Null' 
    final_df['supervisingproviderfname'] = 'Null' 
    final_df['supervisingprovideraddressline2'] = 'Null'
    final_df['diag17'] = 'Null'
    final_df['occurrencecode4'] = 'Null'
    final_df['icdproc2date'] = 'Null' 
    final_df['flex_fld_date_2'] = 'Null' 
    final_df['icdproc21date'] = 'Null'
    final_df['serviceprovidercity'] = 'Null'
    final_df['flex_fld_date_5'] = 'Null'
    final_df['renderingproviderspecialtycode2'] = 'Null' 
    final_df['icdproc21'] = 'Null'
    final_df['etl_certifyingprovider_addr_key'] = 'Null' 
    final_df['etl_supervisingprovider_key'] = 'Null' 
    final_df['icdproc15date'] = 'Null' 
    final_df['period'] = 'Null'
    final_df['attendingproviderid'] = 'Null' 
    final_df['operatingprovidertin'] = 'Null'
    final_df['icdproc3date'] = 'Null'
    final_df['billingprovideraveragelinecharge'] = 'Null'
    final_df['referringproviderspecialtycode1'] = 'Null'
    final_df['certifyingproviderstate'] = 'Null' 
    final_df['icdproc8date'] = 'Null'
    final_df['icdproc16date'] = 'Null'
    final_df['cdx_aux_date_4'] = 'Null' 
    final_df['attendingprovidercity'] = 'Null'
    final_df['cdx_aux_2'] = 'Null' 
    final_df['certifyingproviderfname'] = 'Null' 
    final_df['referringproviderfname'] = 'Null'
    final_df['referringprovidercity'] = 'Null' 
    final_df['serviceproviderspecialtycode3'] = 'Null'
    final_df['icdproc16'] = 'Null' 
    final_df['supervisingprovidertin'] = 'Null' 
    final_df['etl_referringprovider_addr_key'] = 'Null'
    final_df['icdproc6date'] = 'Null'
    final_df['supervisingproviderorg'] = 'Null'
    final_df['etl_referringprovidershist_key'] = 'Null' 
    final_df['icdproc25'] = 'Null' 
    final_df['orderingproviderphonenumber'] = 'Null'
    final_df['cdx_aux_date_5'] = 'Null' 
    final_df['flex_fld_4'] = 'Null' 
    final_df['etl_subscriber_addr_key'] = 'Null' 
    final_df['attendingproviderentitytype'] = 'Null'
    final_df['serviceproviderlname'] = 'Null'
    final_df['flex_fld_num_1'] = 'Null'
    final_df['supervisingprovideraddressline1'] = 'Null'
    final_df['otherprovideraddressline2'] = 'Null'
    final_df['occurrencecode6'] = 'Null'
    final_df['icdproc10'] = 'Null' 
    final_df['icdproc30'] = 'Null'
    final_df['supervisingproviderspecialtycode2'] = 'Null'
    final_df['orderingproviderspecialtycode1'] = 'Null'
    final_df['flex_fld_3'] = 'Null'
    final_df['etl_certifyingprovider_key'] = 'Null'
    final_df['otherproviderorg'] = 'Null' 
    final_df['occurrencecode1'] = 'Null' 
    final_df['attendingproviderspecialtycode2'] = 'Null'
    final_df['icdproc9'] = 'Null'
    final_df['etl_prescribingprovider_addr_key'] = 'Null'
    final_df['diag15'] = 'Null'
    final_df['flex_fld_num_3'] = 'Null'
    final_df['billingproviderspecialtycode3'] = 'Null'
    final_df['icdproc23date'] = 'Null' 
    final_df['diag14'] = 'Null' 
    final_df['final_clm_ind'] = 'Null' 
    final_df['flex_fld_num_4'] = 'Null'
    final_df['icdproc26'] = 'Null' 
    final_df['flex_fld_5'] = 'Null' 
    final_df['attendingproviderorg'] = 'Null' 
    final_df['icdproc27'] = 'Null'
    final_df['operatingproviderorg'] = 'Null' 
    final_df['icdproc29date'] = 'Null' 
    final_df['serviceproviderzipcode'] = 'Null' 
    final_df['icdproc30date'] = 'Null'
    final_df['etl_serviceprovidershist_key'] = 'Null'
    final_df['operatingproviderfname'] = 'Null'
    final_df['referringprovidertin'] = 'Null'
    final_df['etl_operatingprovider_key'] = 'Null'
    final_df['otherproviderid'] = 'Null'
    final_df['icdproc12date'] = 'Null' 
    final_df['operatingprovideraddressline2'] = 'Null'
    final_df['supervisingprovidernpi'] = 'Null'
    final_df['etl_billingprovidershist_key'] = 'Null'
    final_df['billingproviderspecialtycode1'] = 'Null'
    final_df['serviceproviderphonenumber'] = 'Null'
    final_df['serviceproviderspecialtycode1'] = 'Null'
    final_df['icdproc18'] = 'Null'
    final_df['billingproviderspecialtycode2'] = 'Null'
    final_df['orderingprovideraddressline2'] = 'Null' 
    final_df['serviceproviderstate'] = 'Null'
    final_df['diag22'] = 'Null'
    final_df['occurrencecode3'] = 'Null'
    final_df['serviceprovidermname'] = 'Null'
    final_df['attendingprovideraddressline2'] = 'Null'
    final_df['certifyingprovidertaxonomy'] = 'Null' 
    final_df['icdproc23'] = 'Null'
    final_df['serviceprovidernpi'] = 'Null'
    final_df['otherproviderentitytype'] = 'Null'
    final_df['icdproc13'] = 'Null'
    final_df['cdx_aux_num_5'] = 'Null'
    final_df['flex_fld_num_5'] = 'Null' 
    final_df['operatingprovideraddressline1'] = 'Null'
    final_df['icdproc24date'] = 'Null' 
    final_df['etl_referringprovider_key'] = 'Null'
    final_df['referringprovideraddressline1'] = 'Null'
    final_df['attendingproviderspecialtycode1'] = 'Null'
    final_df['otherproviderspecialtycode1'] = 'Null' 
    final_df['attendingproviderphonenumber'] = 'Null' 
    final_df['occurrencecode5'] = 'Null' 
    final_df['referringproviderspecialtycode2'] = 'Null'
    final_df['attendingprovidertin'] = 'Null'
    final_df['cdx_aux_num_1'] = 'Null' 
    final_df['referringprovideraveragelinecharge'] = 'Null'
    final_df['operatingproviderphonenumber'] = 'Null'
    final_df['supervisingprovidercity'] = 'Null' 
    final_df['certifyingproviderspecialtycode1'] = 'Null' 
    final_df['prescribingproviderspecialtycode2'] = 'Null' 
    final_df['orderingproviderfname'] = 'Null'
    final_df['certifyingproviderid'] = 'Null' 
    final_df['otherproviderlname'] = 'Null' 
    final_df['flex_fld_num_2'] = 'Null' 
    final_df['orderingprovidertin'] = 'Null'
    final_df['icdproc8'] = 'Null' 
    final_df['orderingproviderspecialtycode2'] = 'Null'
    final_df['etl_otherprovider_addr_key'] = 'Null' 
    final_df['orderingproviderlname'] = 'Null' 
    final_df['diag25'] = 'Null' 
    final_df['attendingproviderfname'] = 'Null' 
    final_df['icdproc25date'] = 'Null'
    final_df['orderingprovidermname'] = 'Null' 
    final_df['attendingproviderzipcode'] = 'Null'
    final_df['diag24'] = 'Null' 
    final_df['orderingprovidertaxonomy'] = 'Null' 
    final_df['otherproviderspecialtycode3'] = 'Null' 
    final_df['etl_attendingprovidershist_key'] = 'Null' 
    final_df['diag19'] = 'Null' 
    final_df['custsubscriberid'] = 'Null' 
    final_df['cdx_aux_num_3'] = 'Null' 
    final_df['value_code'] = 'Null' 
    final_df['certifyingprovidertin'] = 'Null' 
    final_df['referringprovideraddressline2'] = 'Null' 
    final_df['etl_prescribingprovider_key'] = 'Null'
    final_df['otherproviderspecialtycode2'] = 'Null'
    final_df['prescribingproviderspecialtycode3'] = 'Null'
    final_df['icdproc6'] = 'Null'
    final_df['serviceproviderid'] = 'Null'
    final_df['operatingproviderstate'] = 'Null'
    """
    
    
    
    final_df = final_df.sort_values(by=['ClaimPaidDate'])
    final_df["Date"] = final_df["ClaimPaidDate"]
    final_df["Date"] = final_df["Date"].astype(str)
    final_df["Date"] = final_df["Date"].str[:7]
    out = final_df
    out = out.drop(columns=['Date'])
    
    
    i = 0
    j = 0
    k = 0
    c = ""
    for index, row in final_df.iterrows():
        j+=1
        if not (c ==row["Date"]):
            
            c = row["Date"]
            if k!=0:
                out.iloc[k:j-1].to_csv("Out/"+folder+" Month "+str(i)+'.csv')
            k = j
            i+=1
    print("done")
            

In [3]:
"""order for function
taxonomy 
num_providers
Folder"""

syn_data(22,1000, 'Counseling_Psychology')

#syn_data(22,1000, 'Anesthesiology')

#syn_data(22,1000, 'Nuclear_Radiology')

#syn_data(22,1000, 'Orthopaedic_Surgery')

#syn_data(22,1000, 'Psychiatry_Neurology')



ValueError: Length of values (55) does not match length of index (72)

In [ ]:
print("Done")